# Grant: Per-probe parameter editing + post-run plot/GUI reload

Use this notebook to:
1. edit `grant_recording_config.json` per-probe/per-mode overrides from notebook cells,
2. resolve `ks_dir` and `save_path` for any run mode + probe,
3. reload and rerun:
   - `bc.plot_summary_data(...)` (correct spelling; not `plot_sumary_data`)
   - `bc.compare_manual_vs_bombcell(save_path)`
   - `bc.unit_quality_gui(...)` (main interactive GUI).


In [ ]:
from pathlib import Path
import json
import copy

import bombcell as bc

CONFIG_FILE = Path('../configs/grant_recording_config.json')
RUN_MODE = 'batch'  # batch | single_probe | np20_rerun
TARGET_PROBE = 'A'  # A-F
SAVE_CONFIG = False  # set True only when you want to write config updates


In [ ]:
import sys
from pathlib import Path
sys.path.insert(0, str((Path.cwd() / '..' / '..').resolve()))

from grant.grant_config import load_grant_config

cfg = load_grant_config(CONFIG_FILE)
print('Loaded config:', cfg['config_path'])
print('Recording:', cfg['recording_name'])
print('Target probe:', TARGET_PROBE)
print('Run mode:', RUN_MODE)


## 1) Edit per-probe/per-mode overrides inside notebook

- `probe_param_overrides[probe]['all_modes']` applies to every mode for that probe.
- `probe_param_overrides[probe]['modes'][mode]` applies only for one mode.
- Set `SAVE_CONFIG=True` to persist changes back to JSON.


In [ ]:
raw = json.loads(CONFIG_FILE.read_text(encoding='utf-8'))
probe_cfg = raw.setdefault('probe_param_overrides', {}).setdefault(TARGET_PROBE, {})
all_modes = probe_cfg.setdefault('all_modes', {})
modes = probe_cfg.setdefault('modes', {})
this_mode = modes.setdefault(RUN_MODE, {})

# ---- Example edits (change values as needed) ----
all_modes['minPresenceRatio'] = all_modes.get('minPresenceRatio', 0.7)
this_mode['maxRPVviolations'] = this_mode.get('maxRPVviolations', 0.2)

print('all_modes overrides for probe', TARGET_PROBE, ':', all_modes)
print(f"mode-specific overrides for probe {TARGET_PROBE} / {RUN_MODE}:", this_mode)

if SAVE_CONFIG:
    CONFIG_FILE.write_text(json.dumps(raw, indent=2), encoding='utf-8')
    print('Saved updated config to', CONFIG_FILE)
else:
    print('SAVE_CONFIG=False -> no file written')


## 2) Resolve run paths for this mode/probe

The unified runner stages outputs to:
- `.../kilosort4_{probe}/bombcell/DEFAULT` for `batch`
- `.../kilosort4_{probe}/bombcell/NP2_RERUN` for `np20_rerun`
- `.../kilosort4_{probe}/bombcell/SINGLE_PROBE` for `single_probe`


In [ ]:
mode_to_roots = {
    'batch': (cfg['default_ks_staging_root'], 'DEFAULT'),
    'np20_rerun': (cfg['np20_ks_staging_root'], 'NP2_RERUN'),
    'single_probe': (cfg['bombcell_singleprobe_root'], 'SINGLE_PROBE'),
}

staging_root, save_subdir = mode_to_roots[RUN_MODE]
ks_dir = Path(staging_root) / f'kilosort4_{TARGET_PROBE}'
save_path = ks_dir / 'bombcell' / save_subdir

print('ks_dir:', ks_dir)
print('save_path:', save_path)
print('ks_dir exists:', ks_dir.exists())
print('save_path exists:', save_path.exists())


## 3) Reload everything needed for summary plots + GUI


In [ ]:
# Reload parameter + quality metrics from saved bombcell outputs
param, quality_metrics, _ = bc.load_bc_results(str(save_path))

# Reload template waveforms from staged kilosort directory
_, _, template_waveforms, _, _, _, _ = bc.load_ephys_data(str(ks_dir))

# Recompute unit type labels from loaded outputs
unit_type, unit_type_string = bc.qm.get_quality_unit_type(param, quality_metrics)

print('n units:', len(unit_type))
print('template_waveforms shape:', getattr(template_waveforms, 'shape', None))


## 4) Rerun summary plots for any probe after any Bombcell run


In [ ]:
# Correct API call name: plot_summary_data
bc.plot_summary_data(quality_metrics, template_waveforms, unit_type, unit_type_string, param)


## 5) Reload manual-vs-bombcell comparison


In [ ]:
bc.compare_manual_vs_bombcell(str(save_path))


## 6) Launch main/master Bombcell GUI for this run


In [ ]:
gui = bc.unit_quality_gui(
    ks_dir=str(ks_dir),
    quality_metrics=quality_metrics,
    unit_types=unit_type,
    param=param,
    save_path=str(save_path),
)
gui
